# Settima lezione. Parte B

Importeremo i file da una repositoria su github.

In [1]:
baseURL    = 'https://raw.githubusercontent.com/'
user       = 'domenicozambella/'
repository = 'BioTeIndu19/'
branch     = 'master/'

## Ripasso lezione precedente

La seguente tabella contiene le velocità di reazioni misurate con diverse concentrazioni dell'inibitore: `v` è la velocità, `s` è la concentrazione del substrato, ed `i` è la concentrazione dell'inibitore.

In [2]:
import pandas as pd
file       = 'dati/mm3.csv'
df = pd.read_csv( baseURL + user + repository + branch + file )

In [3]:
df['1/s'] = 1 / df['s']
df['1/v'] = 1 / df['v']

Per fare la linearizzazione di Lineweaver-Burk calcoliamo i reciproci e li aggiungiamo come colonne al dataframe.

### Esplorazione grafica

Prima di fare la regressione lineare facciamo un'esplorazione grafica.

In [4]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
param = dict(width = 700, height = 350,
             tools = 'wheel_zoom, reset,pan, box_zoom, crosshair',
            )

Loading BokehJS ...

Per separare i dati in 5 gruppi (uno per ogni valore di `i`) usiamo il metodo `groupby()`.

Ricordiamo che l'oggetto `gby` contiene coppie: 

    (concentrazione inibitore, dataframe corrispondente)

In [5]:
gby = df.groupby( 'i' )

In [6]:
from numpy import array    
from scipy.stats import linregress

p1 = figure(**param )
for i, sdf in gby:
    m, b, _, _, _ = linregress( sdf['1/s'], sdf['1/v'] )
    xmax = sdf['1/s'].max()     
    x = array( [-2, xmax] )
    y = m * x + b
    p1.line( x, y ) 
show( p1 )

## Migliorare la grafica

Proviamo a differenziare le varie curve con dei colori.

Per associare un colore ad ogni elemento di `gby` viene comodo usare la funzione `zip` che date due liste crea una lista di coppie.  Per esempio, il risultato di

    zip( [1,2,3], [4,5,6] )
è

    [(1,4),(2,5),(3,6)]
    
Il metodo `legend.click_policy` permentte di decidere il comportamento della legenda quando si clicca.

In [7]:
p2  = figure(**param)

palette = [ 'red', 'blue', 'green', 'orange', 'purple']
for c, (i, sdf) in zip( palette, gby ):
    m, b, _, _, _ = linregress( sdf['1/s'], sdf['1/v'] )
    xmax = sdf['1/s'].max()     
    x = array( [-2, xmax] )
    y = m * x + b
    p2.line  (x, y, color=c, legend='inibitor = {}'.format(i) )
p2.legend.location = 'top_left'
p2.legend.click_policy= 'hide'
show( p2 )

## Regressione non lineare

I dati suggeriscono che si tratti di un inibitore misto.

La velocità di reazione $v$ dipende dalla concentrazione di substrato $s$ e dell'inibitore $i$ e da 4 parametri $V_\textrm{max}$, $K_\textrm{m}$, $\delta$, $\beta$ secondo la sequente equazione (copiata acriticamente da internet)

$$
v\ =\ \dfrac{V_\textrm{max}s}{K_\textrm{m} \big(1+i\cdot\delta \big)+s \big(1+i\cdot\beta \big) }
$$

Con un procedimento di interpolazione non lineare si stimino i valori di questi 3 parametri.

Per semplicità trascrivo in Python la funzione qui sopra (per leggibilità non uso la notazione `lambda`)

In [8]:
from scipy.optimize import curve_fit
def mmim( x, Vmax, Km, delta, beta):
    s,i = x
    numeratore   = Vmax * s
    denumeratore = Km * (1 + i*delta) + s * ( 1 + i*beta )
    return numeratore / denumeratore

Si faccia attenzione che il primo valore della funzione è letto come coppia di due valori. Questo è necessario perché `curve_fit()` vuole un'unica variabile per i dati indipendenti.

In [9]:
par, cov = curve_fit( mmim, (df['s'], df['i'] ), df['v'] , p0 = (0.2,0.2, 0, 0))
Vmax, Km, delta, beta = par
par

array([  2.20623347e-01,   3.11419553e-01,   3.01738695e-03,
         2.31186808e-04])

A noi interessano i valori nella diagonale di `cov`. Essendo solo 4 valori potremmo estrarli manualmente, ma possiamo anche fare uso della funzione `diag`.

In [10]:
from numpy import diag
diag(cov)

array([  1.47422219e-04,   3.77937117e-03,   9.13761253e-07,
         2.89455352e-08])

Il diametro $\epsilon$ dell'intervallo di confidenza con livello di confidenza $1-\alpha$ si calcola con la formula $t_{\alpha/2}\cdot$SE. Dove SE ̀e l'errore standard e $t_{\alpha/2}$ ̀e quel valore per cui $\Pr(T<t_{\alpha/2})=\alpha/2$ dove $T$ è una variabile t-Student (con i giusti gradi di libertà)

Qui sotto calcolo il valore $t_{\alpha/2}$.

In [11]:
from scipy.stats import t
alpha  = 0.05                  # 95% confidence interval
n      = len(df)               # number of data points
npar   = 4                     # number of parameters
dof    = n - npar              # number of degrees of freedom
talpha2 =  - t.ppf(1-alpha/2, dof) # student-t value for the confidence level

I valori di SE li calcolo dalla diagonale della matrice di covarianza (basta fare la radice quadrata).

In [12]:
se = diag(cov)**0.5
for i,name in enumerate(['Vmax', 'Km', 'δ', 'β']):
    print(name + ' =  {:.4f} ± {:.4f}'.format(par[i], talpha2*se[i]))

Vmax =  0.2206 ± -0.0255
Km =  0.3114 ± -0.1292
δ =  0.0030 ± -0.0020
β =  0.0002 ± -0.0004


Noto che l'intervallo di confidenza per $\beta$ contiene lo $0$. Quindi non c'è abbastanza evidenza come un inibitore misto.

L'analisi dei residui mette in evidenza un altro problema di quelta tabella di dati.

In [13]:
df['res'] = df['v'] - mmim((df['s'], df['i'] ), Vmax, Km, delta, beta)

L'istogramma dei residui non suggerisce una distribuzione normale.

In [14]:
from numpy import histogram
freq, edges = histogram(df['res'], bins=7)

p2 = figure( **param )
p2.quad(top=freq, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="navy", line_color="white", alpha=0.5,
       )
show( p2 )

Il seguente test di normalità fallisce miseramente. Detto in modo informale: la probabilità che questi residui provengano da una distribuzione normale è $<2\%$.

In [15]:
from scipy.stats import shapiro
shapiro( df['res'] )

(0.9464100003242493, 0.2676467299461365)

Fine

<hr><hr><hr>

La seguente cella importa file di stile HTML (può anche essere ignorata)

In [16]:
import requests
file = 'lezioni/style/custom.css'
from IPython.core.display import HTML
html_style = requests.get( baseURL + user + repository + branch + file ).text
HTML( html_style )